In [19]:
from Optimization.Algorithms.Algorithm import Algorithm
from Numerical_Objects.Vector import Vector

import numpy as np 
from math import sqrt, exp, ceil

In [27]:
class DolphinSearch(Algorithm):

    def __init__(self, N, T1, T2, speed, M, A, e):
        self._Dol = None
        self._k = None
        self._f_k = None
        self._l = None
        self._fit_l = None
        self._TS = np.full((N, N), np.inf)
        self._N = N
        self._T1 = T1
        self._T2 = T2
        self._speed = speed
        self._M = M
        self._A = A
        self._e = e

    @classmethod
    def from_dict(cls, dict_data):
        raise Exception("I don't want to do it and I'll not")

    @classmethod
    def from_json(cls, json_data):
        raise Exception('To be not-json or not to be not-json?')

    def to_dict(self):
        raise Exception('Why?')

    def to_json(self):
        raise Exception('No.')

    @property
    def current_state(self):
        return {}

    @property
    def iterations(self):
        return [self.get_new_solution,
                self.information_transfer,
                self.get_optimal_solution
        ]

    def distance(self, a, b):
        result = 0
        for k in a.keys | b.keys:
            result += (a[k] - b[k]) ** 2
        return sqrt(result)
    
    def initialize(self, f, area, seed = None):
        self._Dol = []
        self._f_k = []
        for i in range(self._N):
            point = {k: np.random.uniform(low, up) for k, (low, up) in area.items()}
            point = Vector(point)
            self._Dol.append(point)
            self._f_k.append(f(point))
        self._k = [dol.copy() for dol in self._Dol]

    def get_new_solution(self, f, area):
        self._l = []
        self._fit_l = []
        area_v = {k: (-1, 1) for k in area.keys()}

        for i in range(self._N):
            V = []
            for m in range(self._M):
                velocity = Vector({k: np.random.uniform(low, up) 
                                   for k, (low, up) in area_v.items()})
                velocity = velocity * (self._speed / velocity.length)
                V.append(velocity)

            sol = [self._Dol[i] + v * t for v in V for t in range(1, self._T1 + 1)]
            sol = [s.constrain(area) for s in sol]
            fit = list(map(f, sol))
        
            idx = np.argmin(fit)
            sol_l = sol[idx]
            self._l.append(sol_l)
            self._fit_l.append(fit[idx])
            
            if self._fit_l[i] < self._f_k[i]:
                self._k[i] = self._l[i].copy()
                self._f_k[i] = self._fit_l[i]
        return self.information_transfer
    
    def information_transfer(self, f, area):
        DD = np.empty((self._N, self._N))
        for i in range(self._N):
            for j in range(self._N):
                DD[i, j] = ceil(self.distance(self._Dol[i], self._Dol[j]) / (self._A * self._speed))

        TS = self._TS 
        for i in range(self._N):
            for j in range(self._N):
                if (self._f_k[j] < self._f_k[i] and TS[i, j] > DD[i, j]):
                    TS[i, j] = DD[i, j]
                    
                TS[i, j] -= 1
                if TS[i, j] == 0:
                    if self._f_k[j] < self._f_k[i]:
                        self._k[i] = self._k[j].copy()
                        self._f_k[i] = self._f_k[j]
                        TS[i, j] = self._T2
                    else:
                        TS[i, j] = self._T2

        return self.get_optimal_solution

    def get_optimal_solution(self, f, area):
        DK = []
        DKL = []
        area_v = {k: (-1, 1) for k in area.keys()}
        new_Dol = []

        for i in range(self._N):
            DK.append(self.distance(self._Dol[i], self._k[i]))
            DKL.append(self.distance(self._l[i], self._k[i]))

        R1 = self._T1 * self._speed

        for i in range(self._N):
            if (0 <= DK[i] and DK[i] <= R1):
                R2 = (1 - (2 / self._e)) * DK[i]
                dol = self._k[i] + (self._Dol[i] - self._k[i])*(R2 / DK[i])
                new_Dol.append(dol.constrain(area))
            elif DK[i] > R1:
                num = (DK[i] / self._f_k[i]) + ((DK[i] - DKL[i]) / self._fit_l[i])
                den = self._e * DK[i] * (1 / (exp(-17) + self._f_k[i]))
                R2 = (1 - (num / den)) * DK[i]
                vec = Vector({k: np.random.uniform(low, up) 
                              for k, (low, up) in area_v.items()})
                dol = self._k[i] + (vec * (R2 / vec.length))
                new_Dol.append(dol.constrain(area))

            if f(new_Dol[i]) < self._f_k[i]:
                self._k[i] = new_Dol[i].copy()
                self._f_k[i] = f(new_Dol[i])
                self._Dol[i] = new_Dol.copy()

        return self.get_new_solution

In [28]:
alg = DolphinSearch(N=6, T1=2, T2=3, speed=2, M=2, A=3, e=2)

def mul(x):
    result = 0
    for val in x.values:
        result += val ** 2 
    return result

area = {'x1': (-10, 10), 'x2': (-10, 10)}
alg.initialize(mul, area)
alg.get_new_solution(mul, area)
alg.information_transfer(mul, area)
alg.get_optimal_solution(mul, area)

<bound method DolphinSearch.get_new_solution of <__main__.DolphinSearch object at 0x7f08bc4e5978>>